构造新的 派生类 异常类

In [162]:
class PersonTypeError(TypeError):
    pass

class PersonValueError(ValueError):
    pass

# 定义公共人员类（基本人员类）

In [163]:
import datetime

# 公共人员类（基本人员类）
class Person:
    _num = 0
    def __init__(self, name, sex, birthday, ident):
        if not (isinstance(name, str)) and sex in ("男", "女"):
            raise PersonTypeError(name, sex)
        try:
            birthday = datetime.date(*birthday)
        except:
            raise PersonValueError("Wrong date:", birthday)
        self._name = name
        self._sex = sex
        self._birthday = birthday
        self._id = ident
        Person._num += 1

    def id(self): return self._id
    def name(self): return self._name
    def sex(self): return self._sex
    def birthday(self): return self._birthday
    def age(self): return (datetime.date.today().year - self._birthday.year)

    # 修改名字的函数
    def set_name(self, name):
        if not isinstance(name, str):
            raise PersonValueError("set_name", name)
        self._name = name

    def __lt__(self, another):
        if not isinstance(another, Person):
            raise PersonTypeError(another)
        return self._id < another._id

    @classmethod
    def num(cls): return Person._num

    #由于字符串join函数要求参数是可迭代对象，因此先构造元组
    def __str__(self):
        return "".join((self._id + " ", self._name + " ", self._sex + " ", str(self._birthday)))

    def details(self):
        return ", ".join((
            "\n"
            "编号：\t\t" + self._id,
            "\n姓名：\t\t" + self._name,
            "\n性别：\t\t" + self._sex,
            "\n出生日期：\t" + str(self._birthday)
        ))


In [164]:
p1 = Person("唐简单", "女", (1995, 7, 15), "1201510111")
p2 = Person("记得是", "男", (1996, 2, 7), "1201510561")
p3 = Person("开会", "男", (1985, 9, 25), "1202350111")
p4 = Person("换衣服", "女", (1991, 1, 11), "1201510199")

In [165]:
plist2 = [p1, p2, p3, p4]
for p in plist2:
    print(p)

1201510111 唐简单 女 1995-07-15
1201510561 记得是 男 1996-02-07
1202350111 开会 男 1985-09-25
1201510199 换衣服 女 1991-01-11


In [166]:
print("\nAfter sorting:")
plist2.sort()
for p in plist2:
    print(p.details())


After sorting:

编号：		1201510111, 
姓名：		唐简单, 
性别：		女, 
出生日期：	1995-07-15

编号：		1201510199, 
姓名：		换衣服, 
性别：		女, 
出生日期：	1991-01-11

编号：		1201510561, 
姓名：		记得是, 
性别：		男, 
出生日期：	1996-02-07

编号：		1202350111, 
姓名：		开会, 
性别：		男, 
出生日期：	1985-09-25


In [167]:
print("people created:", Person.num(), "\n")

people created: 4 



# 定义 学生类（由公共类派生）

In [168]:

class Student(Person):
    _id_num = 0

    @classmethod
    def _id_gen(cls):  #实现学号生成规则
        cls._id_num += 1
        year = datetime.date.today().year
        return "1{:04}{:05}".format(year, cls._id_num)

    def __init__(self, name, sex, birthday, department):
        Person.__init__(self, name, sex, birthday, Student._id_gen())
        self._department = department
        self._enroll_date = datetime.date.today()
        self._courses = {} #一个空字典

    def set_course(self, course_name):
        self._courses[course_name] = None

    def set_score(self, course_name, score):
        if course_name not in self._courses:
            raise PersonValueError("No this course selected:", course_name)
        self._courses[course_name] = score

    def scores(self):
        return [(cname, self._courses[cname]) for cname in self._courses]

    def details(self):
        #更推荐的是使用super()方式调用基类中的属性和方法
        return ", ".join((Person.details(self),
                          "\n入学时间：\t" + str(self._enroll_date),
                          "\n院系：\t\t" + str(self._department),
                          "\n课程记录：\t" + str(self.scores())
                          ))

In [169]:
p5 = Student("张子玉", "女", (1994, 10, 16), department="制造")
p6 = Student("李国林", "男", (1992,5, 12), department="经管")

In [170]:
print(p5)

1201900001 张子玉 女 1994-10-16


In [171]:
print(p5.details())


编号：		1201900001, 
姓名：		张子玉, 
性别：		女, 
出生日期：	1994-10-16, 
入学时间：	2019-12-16, 
院系：		制造, 
课程记录：	[]


In [172]:
p5.set_course("数学")
print(p5.details())


编号：		1201900001, 
姓名：		张子玉, 
性别：		女, 
出生日期：	1994-10-16, 
入学时间：	2019-12-16, 
院系：		制造, 
课程记录：	[('数学', None)]


In [173]:
p5.set_score("数学", 118)
print(p5.details())


编号：		1201900001, 
姓名：		张子玉, 
性别：		女, 
出生日期：	1994-10-16, 
入学时间：	2019-12-16, 
院系：		制造, 
课程记录：	[('数学', 118)]


# 教工类的实现

In [174]:

# 教工类的实现
class Staff(Person):
    _id_num = 0

    @classmethod
    def _id_gen(cls, birthday):  #实现职工号生成规则
        cls._id_num += 1
        birth_year = datetime.date(*birthday).year
        return "0{:04}{:05}".format(birth_year, cls._id_num)

    def __init__(self, name, sex, birthday, entry_date=None):
        super().__init__(name, sex, birthday, Staff._id_gen(birthday))

        if entry_date:
            try:
                self._entry_date = datetime.date(*entry_date)
            except:
                raise PersonValueError("Wrong date：", entry_date)
        else:
            self._entry_date = datetime.date.today()

        self._salary = 2000 #设置默认最低工资
        self._department = "未定"
        self._positon = "未定"

    def set_salary(self, amount):
        if not type(amount) is int:
            raise TypeError
        self._salary = amount

    def set_position(self, position):
        self._positon = position
    def set_department(self, department):
        self._department = department

    def details(self):
        return ", ".join((
            super().details(),
            "\n入职日期：\t" + str(self._entry_date),
            "\n院系：\t\t" + str(self._department),
            "\n职位：\t\t" + str(self._positon),
            "\n工资：\t\t" + str(self._salary)
        ))

In [175]:
p7 = Staff("浮动板", "女", (1974, 10, 16))
p8 = Staff("人多少", "男", (1942,5, 12))

In [176]:
print(p7)
print(p8)

0197400001 浮动板 女 1974-10-16
0194200002 人多少 男 1942-05-12


In [177]:
p7.set_department("数学")
p7.set_position("副教授")
p7.set_salary(12000)

In [178]:
print(p7.details())
print(p8.details())


编号：		0197400001, 
姓名：		浮动板, 
性别：		女, 
出生日期：	1974-10-16, 
入职日期：	2019-12-16, 
院系：		数学, 
职位：		副教授, 
工资：		12000

编号：		0194200002, 
姓名：		人多少, 
性别：		男, 
出生日期：	1942-05-12, 
入职日期：	2019-12-16, 
院系：		未定, 
职位：		未定, 
工资：		2000
